# Learning Encodes Alignment within a Protein Family.

In [1]:
# Needed to import modules from helpers
import sys
import os

current_dir = os.getcwd()
# Gehe einen Ordner nach oben
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [6]:
import torch
import esm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.cm as cm
from helpers import helper
import time
import io
import urllib3
import requests

# kleineres Modell 'esm2_t6_8M_UR50D' zum testen 
# verwendet 36-layer Transformer trained on UniParc" (ca. 670 Mio. Parameter ) im Paper.
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

if torch.cuda.is_available():
    model = model.cuda()
    print("Modell auf GPU geladen.")

### Motivation

One of the oldest assumptions of sequencing biology: The underlying structure of a protein is a hidden variable that influences the patterns observed in sequence data. And vice versa the patterns observed in the sequence data influence the dtructure of a protein.
In short: Structural information is encoded in the sequences.

- secondary structure decides local choice and order of sequences
- tertiary decides over long range choice and order of sequences

Underlying general Hypothesis: Since 3d struture is encoded in the sequences. It is a logical hypothesis that via unsupervised learning the model learns to decode the hidden information about the secondary and tertiary strucure of the protein implicitly. 

In the paper they start by using simple linear models on top of the learned respresentations to see whether or not even simple models can infer about structure using the learned representations. If they are able to do that that would be very impressive.

enabling a direct inspection of the structural content of representations.

By comparing representations of the Transformer before and after pretraining, we can identify the information that emerges as a result of the unsupervised learning

fivefold cross validation experiment to study generalization of structural information at the family, superfamily, and fold level.
-	For each of the three levels, we construct a dataset of 15,297 protein structures using the SCOPe database.

### Hypothesis 1:


### Hypothesis 2:
final hidden representations of a sequence encode information about the family it belongs to.

### Method:

- Get Dataset (Pfam)
- compare the distribution of cosine similarities of representations between pairs of residues that are aligned in the family’s MSA background distribution of cosine similarities between unaligned pairs of residues.
- Compare with distributions befor learning (We need the embeddings befor pretraining (randomize model))
